In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

Num GPUs Available:  1


In [3]:
from bert.model import create_albert_model
model = create_albert_model(model_dimension=512,
                            transformer_dimension=512 * 4,
                            num_attention_heads=512 // 64,
                            num_transformer_layers=6,
                            vocab_size=22,
                            dropout_rate=0.,
                            max_relative_position=128)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 512)    11264       input_1[0][0]                    
__________________________________________________________________________________________________
transformer (Transformer)       (None, None, 512)    2906176     embedding[0][0]                  
                                                                 transformer[0][0]                
                                                                 transformer[1][0]                
                                                                 transformer[2][0]            

In [10]:
from bert.optimizers import (ECE, masked_sparse_categorical_crossentropy,
                             BertLinearSchedule)

opt = tfa.optimizers.AdamW(learning_rate=1E-5,
                           beta_2=0.98,
                           epsilon=1E-6,
                           weight_decay=0)

# opt = tf.optimizers.Adam(learning_rate=1E-4,
#                            beta_2=0.98,
#                            epsilon=1E-6)

opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)

true_labels = tf.keras.layers.Input(
    shape=(None,), dtype=tf.int32, batch_size=None)

model.compile(
    target_tensors=true_labels,    
    loss=masked_sparse_categorical_crossentropy,
    metrics=[ECE],
    optimizer=opt,
    experimental_run_tf_function=True)

In [11]:
callbacks = [
    BertLinearSchedule(1E-5, 1000, int(1E7)),
#     tf.keras.callbacks.ModelCheckpoint(filepath='jupyter_test_checkpoints/ckpt_{epoch}_{val_ECE:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(
        log_dir='../uniparc_checkpoints/tblogs/test',
        histogram_freq=0,
        write_graph=False,
        profile_batch=0,
        update_freq='epoch',
        embeddings_freq=0)]    

In [12]:
from bert.dataset import create_masked_input_dataset

training_data = create_masked_input_dataset(
    sequence_path='../uniparc_data/sequences_train.txt',
    max_sequence_length=128,
    batch_size=32)

training_data = training_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

valid_data = create_masked_input_dataset(
    sequence_path='../uniparc_data/sequences_valid.txt',
    max_sequence_length=128,
    batch_size=32)

valid_data = valid_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [13]:
model.fit(training_data, steps_per_epoch=500, epochs=5,
          verbose=1, validation_data=valid_data, validation_steps=10,
          callbacks=callbacks)

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


500/500 [==============================] - 48s 96ms/step - loss: 2.9327 - ECE: 18.9452 - val_loss: 2.7340 - val_ECE: 15.4058
Epoch 2/5
500/500 [==============================] - 37s 75ms/step - loss: 2.7315 - ECE: 15.3678 - val_loss: 2.7061 - val_ECE: 14.9736
Epoch 3/5
500/500 [==============================] - 37s 75ms/step - loss: 2.7047 - ECE: 14.9613 - val_loss: 2.7082 - val_ECE: 15.0145
Epoch 4/5
500/500 [==============================] - 37s 75ms/step - loss: 2.6917 - ECE: 14.7676 - val_loss: 2.7021 - val_ECE: 14.9172
Epoch 5/5
500/500 [==============================] - 37s 75ms/step - loss: 2.6851 - ECE: 14.6712 - val_loss: 2.6891 - val_ECE: 14.7309


In [ ]:
model.save('../uniparc_checkpoints/test.h5')